In [20]:
import pandas as pd
import defillama_utils as dfl
import pandas_utils as putil
from datetime import timedelta

# df = pd.read_csv('csv_outputs/latest_tvl_app_trends_by_token.csv')

In [21]:
name = 'arrakis-finance'
chain = 'Optimism'
trailing_num_days = 90

In [22]:
min_dt_ts = putil.get_unix_timestamp(trailing_num_days)
print(min_dt_ts)
df = dfl.get_single_tvl(name, [chain], min_dt_ts)
df['price_usd'] = df['usd_value']/df['token_value']
display(df)

1668574800


,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,price_usd
0,2022-01-20 00:00:00,AELIN,8.689625e+01,5.436925e+06,1.089262e+07,arrakis-finance,Optimism,Yield,62568.001085
1,2022-01-20 00:00:00,WETH,1.755789e+03,5.455676e+06,1.089262e+07,arrakis-finance,Optimism,Yield,3107.249995
2,2022-01-20 00:00:00,USDC,1.986700e-01,1.986700e-01,1.089262e+07,arrakis-finance,Optimism,Yield,1.000000
3,2022-01-20 00:00:00,SNX,2.864680e+00,1.412289e+01,1.089262e+07,arrakis-finance,Optimism,Yield,4.930006
4,2022-01-20 00:00:00,SUSD,2.122160e+00,2.121020e+00,1.089262e+07,arrakis-finance,Optimism,Yield,0.999463
...,...,...,...,...,...,...,...,...,...
7405,2023-02-14 17:59:35,HAN,1.723814e+03,1.258384e+04,7.107073e+06,arrakis-finance,Optimism,Yield,7.300000
7406,2023-02-14 17:59:35,POP,3.199872e+04,8.420783e+03,7.107073e+06,arrakis-finance,Optimism,Yield,0.263160
7407,2023-02-14 17:59:35,L2DAO,3.870414e+06,8.282918e+03,7.107073e+06,arrakis-finance,Optimism,Yield,0.002140
7408,2023-02-14 17:59:35,KWENTA,8.528807e+02,2.679154e+05,7.107073e+06,arrakis-finance,Optimism,Yield,314.129998


In [23]:
df['token_rank_desc_prot'] = df.groupby(['chain','token','protocol'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
latest_prices_df_raw_prot = df[~df['price_usd'].isna()][['token','chain','protocol','price_usd']][df['token_rank_desc_prot'] ==1]
latest_prices_df_prot = latest_prices_df_raw_prot.groupby(['token','chain','protocol']).median('price_usd')
latest_prices_df_prot = latest_prices_df_prot.reset_index()
print(latest_prices_df_prot)
latest_prices_df_prot = latest_prices_df_prot.rename(columns={'price_usd':'latest_price_usd'})

df = df.merge(latest_prices_df_prot,on=['token','chain','protocol'], how='left')

df['value_usd_latest_price'] = df['token_value'] * df['latest_price_usd']

# Calc DoD flow differences and flow differences since the beginning, calc'd at latest price

display(df)

# piv_df = ndf.groupby(['chain','protocol','token']).agg({'net_dollar_flow_latest_price':'sum'})
# piv_df['share_of_total'] = piv_df['net_dollar_flow_latest_price'] / piv_df.groupby(['chain','protocol'])['net_dollar_flow_latest_price'].transform('sum')


# display(piv_df)

     token     chain         protocol    price_usd
0    AELIN  Optimism  arrakis-finance  1856.769356
1      DAI  Optimism  arrakis-finance     0.998972
2     GYSR  Optimism  arrakis-finance     0.055336
3      HAN  Optimism  arrakis-finance     7.300000
4   KWENTA  Optimism  arrakis-finance   314.129998
5    L2DAO  Optimism  arrakis-finance     0.002140
6     LYRA  Optimism  arrakis-finance     0.165496
7     MNYE  Optimism  arrakis-finance   100.520159
8       OP  Optimism  arrakis-finance     2.370000
9      POP  Optimism  arrakis-finance     0.263160
10      QI  Optimism  arrakis-finance     0.117730
11     SNX  Optimism  arrakis-finance     2.450000
12     STG  Optimism  arrakis-finance     0.812590
13    SUSD  Optimism  arrakis-finance     1.000000
14  THALES  Optimism  arrakis-finance     0.836038
15    USDC  Optimism  arrakis-finance     0.998491
16    WETH  Optimism  arrakis-finance  1542.580002
17  WSTETH  Optimism  arrakis-finance  1709.789986


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_68298/2923573723.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  latest_prices_df_raw_prot = df[~df['price_usd'].isna()][['token','chain','protocol','price_usd']][df['token_rank_desc_prot'] ==1]


,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,price_usd,token_rank_desc_prot,latest_price_usd
0,2022-01-20 00:00:00,AELIN,8.689625e+01,5.436925e+06,1.089262e+07,arrakis-finance,Optimism,Yield,62568.001085,390,1856.769356
1,2022-01-20 00:00:00,WETH,1.755789e+03,5.455676e+06,1.089262e+07,arrakis-finance,Optimism,Yield,3107.249995,390,1542.580002
2,2022-01-20 00:00:00,USDC,1.986700e-01,1.986700e-01,1.089262e+07,arrakis-finance,Optimism,Yield,1.000000,390,0.998491
3,2022-01-20 00:00:00,SNX,2.864680e+00,1.412289e+01,1.089262e+07,arrakis-finance,Optimism,Yield,4.930006,390,2.450000
4,2022-01-20 00:00:00,SUSD,2.122160e+00,2.121020e+00,1.089262e+07,arrakis-finance,Optimism,Yield,0.999463,390,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
7405,2023-02-14 17:59:35,HAN,1.723814e+03,1.258384e+04,7.107073e+06,arrakis-finance,Optimism,Yield,7.300000,1,7.300000
7406,2023-02-14 17:59:35,POP,3.199872e+04,8.420783e+03,7.107073e+06,arrakis-finance,Optimism,Yield,0.263160,1,0.263160
7407,2023-02-14 17:59:35,L2DAO,3.870414e+06,8.282918e+03,7.107073e+06,arrakis-finance,Optimism,Yield,0.002140,1,0.002140
7408,2023-02-14 17:59:35,KWENTA,8.528807e+02,2.679154e+05,7.107073e+06,arrakis-finance,Optimism,Yield,314.129998,1,314.129998
